In [6]:
import torch.nn as nn
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm_
torch.autograd.set_detect_anomaly(True)
import websocket, json, pprint, talib, numpy
from binance.client import Client
from binance.enums import *
import config
import joblib

In [2]:
data = pd.read_csv('data_1m.csv')

close = data.loc[:, "labels"]
data.head()

,opens,highs,lows,closes,volumes,qavs,numTrades,tbbavs,tbqavs,rsis,obvs,labels
0,-0.647322,-0.652692,-0.652362,-0.664268,-0.324874,-0.369299,-0.298627,-0.400240,-0.431825,-0.769706,-0.873230,0
1,-0.655584,-0.660894,-0.667964,-0.676176,-0.078757,-0.145297,-0.198432,-0.337637,-0.375243,-0.660974,-0.966605,0
2,-0.680247,-0.683287,-0.684137,-0.695977,0.153818,0.065629,-0.029352,-0.258902,-0.304005,-0.802457,-0.890796,1
3,-0.696725,-0.675500,-0.684708,-0.663745,-0.142640,-0.204205,-0.298627,-0.212348,-0.261596,-0.373340,-0.512582,0
4,-0.666631,-0.678203,-0.669414,-0.681284,-0.466697,-0.500130,-0.542854,-0.359751,-0.395499,-0.780812,-0.831478,0


In [3]:
data = data.to_numpy()
close = close.to_numpy()
extra = np.shape(data)[0] % 32
data = data[extra:, :]
data = torch.tensor(data)
data = data.to(torch.float)
close = close[extra:]
close = torch.tensor(close)
close = close.to(torch.float)
data, X_test, close, y_test = train_test_split(data, close, test_size=0.5, shuffle = False)

class Dataset():
    def __init__(self, array, labels):
        self.array = array
        self.labels = labels

    def __len__(self):
        return self.array.shape[0]

    def __getitem__(self, idx):
        return self.array[idx, :], self.labels[idx]

In [4]:

batch_size = 16 
fulldata = Dataset(data, close)
dataloader = DataLoader(fulldata, batch_size)
loss_function = nn.BCELoss()

class RNNModel(torch.nn.Module):
    def __init__(self, input_size, HL_size, num_layers = 1):
        super(RNNModel, self).__init__()
        self.rnn = torch.nn.RNN(input_size=input_size,
                                hidden_size=HL_size,
                                num_layers=num_layers)
        
        self.linear = torch.nn.Linear(HL_size , 12)
        self.linear2 = torch.nn.Linear(12, 1)
        self.num_layers = num_layers
        self.hidden_size = HL_size
        
    def forward(self, inputs, hidden):
        out, hidden = self.rnn(inputs, hidden)
        pred = self.linear(out)
        pred = self.linear2(pred)
        pred = torch.sigmoid(pred)
        return pred, hidden
    def init_hidden(self, batch = 1):
        hidden = torch.zeros(self.num_layers, batch, self.hidden_size)
        return hidden

def rnn_train():
    model = RNNModel(np.shape(data)[1], HL_size = 24, num_layers = 2)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
    hidden = model.init_hidden(batch_size)
    for batch_idx, (batch, truth) in enumerate(dataloader):
        optimizer.zero_grad()
        hidden = hidden.detach()
        batch = np.reshape(batch, (1, -1, 12))
        pred, hidden = model.forward(batch, hidden)
        pred = torch.squeeze(pred, 0)
        pred = torch.squeeze(pred, 1)
        single_loss = loss_function(pred, truth)
        single_loss.backward(retain_graph=True)
        clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(batch_idx)
            #for name, param in model.named_parameters():
                #if param.requires_grad:
                    #print(name, param.data)
            print(single_loss)

    return model, hidden
            
trained_model, final_hidden = rnn_train()

0
tensor(0.6861, grad_fn=<BinaryCrossEntropyBackward0>)
100
tensor(0.0004, grad_fn=<BinaryCrossEntropyBackward0>)
200
tensor(0.2770, grad_fn=<BinaryCrossEntropyBackward0>)
300
tensor(0.0031, grad_fn=<BinaryCrossEntropyBackward0>)
400
tensor(9.0159e-05, grad_fn=<BinaryCrossEntropyBackward0>)


In [8]:
testdata = Dataset(X_test, y_test)
testdataloader = DataLoader(testdata, batch_size)
hidden = final_hidden
total_batches = 0
total_error = 0

model = RNNModel(np.shape(data)[1], HL_size = 24, num_layers = 2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
    
for batch_idx, (batch, truth) in enumerate(testdataloader):
    optimizer.zero_grad()
    hidden = hidden.detach()
    batch = np.reshape(batch, (1, -1, 12))
    pred, hidden = model.forward(batch, hidden)
    pred = torch.squeeze(pred, 0)
    pred = torch.squeeze(pred, 1)
    single_loss = loss_function(pred, truth)
    single_loss.backward(retain_graph=True)
    clip_grad_norm_(model.parameters(), 5)
    optimizer.step()
    total_batches += 16
    
    total_error += np.sum(np.abs(pred.detach().numpy() - truth.detach().numpy()))

        
print(total_error/total_batches)

0.037004357929257284


# MODEL PREDICTION

In [7]:
SOCKET = "wss://stream.binance.com:9443/ws/ethusdt@kline_1m"

RSI_PERIOD = 14
TRADE_SYMBOL = 'ETHUSD'

in_position = False

client = Client(config.KEY, config.SECRET, tld='us')

scaler = joblib.load('standard_scaler_1m.save')

In [12]:
opens = []
closes = []
highs = []
lows = []
volumes = []
qavs = []
numTrades = []
tbbavs = []
tbqavs = []
rsis = []
obvs = []

# def order(side, quantity, symbol,order_type=ORDER_TYPE_MARKET):
#     try:
#         print("sending order")
#         order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
#         print(order)
#     except Exception as e:
#         print("an exception occured - {}".format(e))
#         return False

#     return True

    
def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('closed connection')

def on_message(ws, message):
    global closes, in_position
    
    #print('received message')
    json_message = json.loads(message)
    #pprint.pprint(json_message)

    candle = json_message['k']

    is_candle_closed = candle['x']
    close = float(candle['c'])
    open_ = float(candle['o'])
    high = float(candle['h'])
    low = float(candle['l'])
    vol = float(candle['v'])
    qav = float(candle['q'])
    numTrade = float(candle['n'])
    tbbav = float(candle['V'])
    tbqav = float(candle['Q'])
    

    if is_candle_closed:
        print("candle closed at {}".format(close))
        closes.insert(0, close)
        volumes.insert(0, vol)
        
#         opens.append(float(open_))
#         highs.append(float(high))
#         lows.append(float(low))
#         qavs.append(float(qav))
#         numTrades.append(float(numTrade))
#         tbbavs.append(float(tbbav))
#         tbqavs.append(float(tbqav))
        
        print("closes")
        print(closes)

        if len(closes) > RSI_PERIOD:
            np_closes = numpy.array(closes[:15])
            np_vols = numpy.array(volumes[:15])
            rsi = talib.RSI(np_closes, RSI_PERIOD)
            obv = talib.OBV(np_closes, np_vols)
            print("all rsis calculated so far")
            print(rsi)
            print("all obvs calculated so far")
            print(obv)
            last_rsi = rsi[-1]
            last_obv = obv[-1]
            print("the current rsi is {}".format(last_rsi))
            
            df = pd.DataFrame(data={"opens": [open_], "highs": [high], "lows": [low], "closes": [close], "volumes": [vol], "qavs": [qav], "numTrades": [numTrade], "tbbavs": [tbbav], "tbqavs": [tbqav], "rsis": [last_rsi],"obvs": [last_obv]})
            scaled_df = scaler.transform(df)
            print(scaled_df)
            data = scaled_df.to_numpy()
            data = torch.tensor(data)
            data = data.to(torch.float)
            
            
            
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()

opened connection


False

In [10]:
df = pd.DataFrame(data={"opens": [1], "highs": [1], "lows": [1], "closes": [1], "volumes": [1], "qavs": [1], "numTrades": [1], "tbbavs": [1], "tbqavs": [1], "rsis": [1],"obvs": [1]})
print(df)
data = df.to_numpy()
data = torch.tensor(data)
data = data.to(torch.float)

print(model(data, hidden))

   opens  highs  lows  closes  volumes  qavs  numTrades  tbbavs  tbqavs  rsis  \
0      1      1     1       1        1     1          1       1       1     1   

   obvs  
0     1  


RuntimeError: For unbatched 2-D input, hx should also be 2-D but got 3-D tensor

In [6]:
torch.save(model.state_dict(), 'rnn_model_1m')

In [9]:
for param in model.state_dict():
    print(param)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
linear.weight
linear.bias
linear2.weight
linear2.bias


In [7]:
class LSTM(nn.Module):
    def __init__(self, hidden_size = 24, input_size=12, output_size=1, num_layers = 1):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers)
        self.linear = nn.Linear(self.hidden_size, output_size)
        
        
    def forward(self, input_seq, hidden):
        lstm_out, self.hidden_cell = self.lstm(input_seq, hidden)
        predictions = torch.sigmoid(self.linear(lstm_out))
        
        return np.squeeze(predictions[-1, :])
    def init_hidden(self, batch = 1):
        hidden = (torch.zeros(self.num_layers, batch, self.hidden_size),
                 torch.zeros(self.num_layers, batch, self.hidden_size))
        
        return hidden

def lstm_train():
    model = LSTM()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    hidden = model.init_hidden(batch_size)
    for batch_idx, (batch, truth) in enumerate(dataloader):
        optimizer.zero_grad()
        hidden = tuple([h.detach() for h in hidden])
        batch = np.reshape(batch, (1, -1, 12))

        y_pred = model(batch, hidden)

        single_loss = loss_function(y_pred, truth)
        single_loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            #for name, param in model.named_parameters():
                #if param.requires_grad:
                    #print(name, param.data)
            print(single_loss)


    return model

lstm_train()

tensor(0.6920, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0165, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0834, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0015, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0009, grad_fn=<BinaryCrossEntropyBackward>)


LSTM(
  (lstm): LSTM(12, 24)
  (linear): Linear(in_features=24, out_features=1, bias=True)
)